Paso 0. Cargue archivos pkl (validación)

In [1]:
import pickle

ruta_archivo = "../files/grading/x_train.pkl"

with open(ruta_archivo, "rb") as file:
    contenido1 = pickle.load(file)


c:\Users\JUAN\anaconda3\envs\envML\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
ruta_archivo = "../files/grading/y_train.pkl"

with open(ruta_archivo, "rb") as file:
    contenido2 = pickle.load(file)

In [3]:
ruta_archivo = "../files/grading/x_test.pkl"

with open(ruta_archivo, "rb") as file:
    contenido3 = pickle.load(file)

In [4]:
ruta_archivo = "../files/grading/y_test.pkl"

with open(ruta_archivo, "rb") as file:
    contenido4 = pickle.load(file)

Paso 1. Limpieza

In [5]:
import pandas as pd

data_train = pd.read_csv('../files/input/train_data.csv.zip', index_col = False, compression = "zip")
data_test = pd.read_csv("../files/input/test_data.csv.zip", index_col = False, compression = "zip")

print(data_train.isnull().sum())

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64


In [6]:
import numpy as np

data_train.rename(columns={"default payment next month": "default"}, inplace=True)
data_test.rename(columns={"default payment next month": "default"}, inplace=True)
data_train.drop(columns=['ID'], inplace=True)
data_test.drop(columns=['ID'], inplace=True)

data_train['EDUCATION'] = data_train['EDUCATION'].apply(lambda x: x if x > 0 else np.nan)
data_test['EDUCATION'] = data_test['EDUCATION'].apply(lambda x: x if x > 0 else np.nan)

data_train['MARRIAGE'] = data_train['MARRIAGE'].apply(lambda x: x if x > 0 else np.nan)
data_test['MARRIAGE'] = data_test['MARRIAGE'].apply(lambda x: x if x > 0 else np.nan)

data_train['EDUCATION'] = data_train['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
data_test['EDUCATION'] = data_test['EDUCATION'].apply(lambda x: 4 if x > 4 else x)

data_train.dropna(inplace=True)
data_test.dropna(inplace=True)


Paso 2. División datasets 

In [7]:
#particionamiento de los datos
x_train = data_train.drop(columns=['default'])
y_train = data_train["default"]
x_test = data_test.drop(columns=['default'])
y_test = data_test["default"]


Paso 3. Creación pipeline 

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC
from sklearn.decomposition import PCA

categorical_features = ['SEX', 'EDUCATION', 'MARRIAGE']
numerical_features = list(set(x_test.columns) - set(categorical_features))

preprocesamiento = ColumnTransformer(
    transformers=[
        ('categoricas', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('numericas', StandardScaler(), numerical_features)
    ],
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocesamiento),
    ('pca', PCA(n_components = None)),
    ('selector', SelectKBest(f_classif)),
    ('svc', SVC(random_state=97))
])


Paso 4. Optimización parametros con cv

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

param_grid = {'pca__n_components': [None],
                'selector__k': [17],
                'svc__gamma': ['scale', 'auto', 0.01, 0.1, 1, 10]}
              
modelo = GridSearchCV(pipeline, param_grid, cv=10, scoring='balanced_accuracy', n_jobs=-1)

modelo.fit(x_train, y_train)


Paso 5. Guardar el modelo

In [23]:
import os
import gzip
import pickle

dir_path = '../files/models'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(modelo, f)
else:
    with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(modelo, f)

Paso 6. Cálculo de metricas 

In [24]:
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score
import json

y_train_pred = modelo.predict(x_train)
y_test_pred = modelo.predict(x_test)

train_metrics = {
    "type": "metrics", 
    'dataset': 'train',
    'precision': precision_score(y_train, y_train_pred),
    'balanced_accuracy': balanced_accuracy_score(y_train, y_train_pred),
    'recall': recall_score(y_train, y_train_pred),
    'f1_score': f1_score(y_train, y_train_pred)
}


test_metrics = {
    "type": "metrics", 
    'dataset': 'test',
    'precision': precision_score(y_test, y_test_pred),
    'balanced_accuracy': balanced_accuracy_score(y_test, y_test_pred),
    'recall': recall_score(y_test, y_test_pred),
    'f1_score': f1_score(y_test, y_test_pred)
}

output_path = '../files/output/metrics.json'
os.makedirs(os.path.dirname(output_path), exist_ok=True) 

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(train_metrics, f, ensure_ascii=False) 
    f.write('\n')
    json.dump(test_metrics, f, ensure_ascii=False) 
    f.write('\n')

Paso 7. Cálculo de matrices de confusión

In [25]:
from sklearn.metrics import confusion_matrix

train_cm = confusion_matrix(y_train, y_train_pred)
test_cm = confusion_matrix(y_test, y_test_pred)

train_cm_dict = {
    'type': 'cm_matrix',
    'dataset': 'train',
    'true_0': {'predicted_0': int(train_cm[0, 0]), 'predicted_1': int(train_cm[0, 1])},
    'true_1': {'predicted_0': int(train_cm[1, 0]), 'predicted_1': int(train_cm[1, 1])}
}

test_cm_dict = {
    'type': 'cm_matrix',
    'dataset': 'test',
    'true_0': {'predicted_0': int(test_cm[0, 0]), 'predicted_1': int(test_cm[0, 1])},
    'true_1': {'predicted_0': int(test_cm[1, 0]), 'predicted_1': int(test_cm[1, 1])}
}

output_path = '../files/output/metrics.json'

with open(output_path, 'a', encoding='utf-8') as f:
    json.dump(train_cm_dict, f, ensure_ascii=False) 
    f.write('\n')
    json.dump(test_cm_dict, f, ensure_ascii=False)  
    f.write('\n')

In [28]:
#prueba
print(modelo.best_params_)
print(modelo.best_score_)
print(modelo.best_estimator_)
print(modelo.best_index_)

{'pca__n_components': None, 'selector__k': 17, 'svc__gamma': 0.1}
0.6502443499687053
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categoricas',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['SEX', 'EDUCATION',
                                                   'MARRIAGE']),
                                                 ('numericas', StandardScaler(),
                                                  ['PAY_6', 'BILL_AMT3',
                                                   'PAY_AMT6', 'PAY_3',
                                                   'BILL_AMT1', 'PAY_AMT2',
                                                   'BILL_AMT6', 'BILL_AMT4',
                                                   'LIMIT_BAL', 'PAY_4',
                                                   'PAY_2', 'PAY_AMT5', 'PAY_5',
                                                   'PAY_0'